In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
from fluidvec import *

In [4]:
from fluidvec.dataset import TrainDataset, get_dataloader

In [5]:
import pickle
with open("../data/train_items/train_items_001.pkl", "rb") as fin:
    items = pickle.load(fin)

In [6]:
vs = VocabSet.load()

In [7]:
loader = get_dataloader("../data/train_items/train_items_001.pkl")

In [8]:
batch = next(iter(loader))

In [9]:
items[0]

({'compos': [2, 3, 4, 5], 'chars': [2, 3], 'word': 4},
 [{'compos': [1], 'chars': [1], 'word': 1},
  {'compos': [1], 'chars': [1], 'word': 1},
  {'compos': [0], 'chars': [0], 'word': 0},
  {'compos': [1], 'chars': [1], 'word': 1}])

In [10]:
model = FluidVecSG(len(vs.word_vocab), len(vs.char_vocab), len(vs.compo_vocab))

../src\fluidvec\model.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.weights = torch.tensor(weights)


In [11]:
items[0][0]["word"]

4

In [12]:
vec_dict = model.transform_batch_data(batch)
tgt = vec_dict["tgt"]
ctx = vec_dict["ctx"]
tgt.size(), ctx.size()

(torch.Size([8, 300]), torch.Size([8, 4, 300]))

In [13]:
noise = model(batch)

In [17]:
noise_vec = model.word_emb(noise)

In [18]:
noise_vec.shape

torch.Size([8, 4, 10066, 300])

In [27]:
log_target = (tgt.unsqueeze(1) * ctx).sum(2).sigmoid().log().sum()

tensor(-110.8461, grad_fn=<SumBackward0>)

In [34]:
(tgt.unsqueeze(1).unsqueeze(1) * noise_vec).neg().sum(3).sigmoid().log().sum()

tensor(-inf, grad_fn=<SumBackward0>)

In [49]:
((tgt.unsqueeze(1).unsqueeze(1) * noise_vec).neg().sum(3).sigmoid()+1e-32).log().sum()

tensor(-4551435.5000, grad_fn=<SumBackward0>)